In [18]:
import os
import datetime
from jinja2 import Template, Environment, FileSystemLoader
from PIL import Image

In [19]:
file_path = "chat.txt"
with open(file_path, mode='r', encoding="utf8") as f:
    data = f.readlines()

In [20]:
#utente = input("Inserisci il received: ")
user =data[0].split(":")[2].split("]")[1][1:]
user

'Jack'

In [21]:
dataset = data [1:]
cleaned_data = []
lista = []

for line in dataset:
    # Check, whether it is a new line or not
    # If the following characters are in the line -> assumption it is NOT a new line
    if '/' in line and ':' in line and ',' in line and '[' in line:
        # grab the info and cut it out
        date = line.split("[")[1].split(",")[0]
        line2 = line[len(date):]
        time = line2.split(",")[1].split("]")[0]
        line3 = line2[len(time):]
        sender = line3.split("]")[1].split(":")[0]
        line4 = line3[len(sender):]
        message = line4[5:-1]
        message = message.replace(" \u200e", "" ).replace("\n", "")
        position = 'received'
        if sender[1:] == user:
            position = 'sent'
        cleaned_data.append([position, date, time[1:], sender[1:], message])
        lista.append(date)
    # else, assumption -> new line. Append new line to previous 'message'
    else:
        new = cleaned_data[-1][-1] + " " + line
        cleaned_data[-1][-1] = new

In [22]:
f.close()

In [23]:
file_html_path = "index_wa.html"
if os.path.exists(file_html_path):
    os.remove(file_html_path) 
i = open(file_html_path, mode='x', encoding="utf8")


In [24]:
file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader)
start=env.get_template("start.txt")
end=env.get_template("end.txt")
day_template=env.get_template("day_template.txt")
message_template=env.get_template("message_template.txt")
media_template=env.get_template("media_template.txt")

In [25]:
i.write(start.render(name=user))

1314

In [26]:
message_date = ""

for m in cleaned_data:
    mess = m[4]
    
    if m[1] != message_date:
        message_date = m[1]
        i.write((day_template.render(day=m[1])))
    
    #posizione conterrà l'inizio della stringa "<allegato:"
    position = m[4].find("<allegato:")
    
    if position>-1:
        if(m[4].find(".jpg")>-1):
            filename = m[4][position+11:len(m[4])-1]
            with Image.open(filename) as image:
                width, height = image.size
            print(str(width) + " " + str(height))
   
            mess =  "<a href=" + m[4][position+11:len(m[4])-1]+ " data-lightbox=" + m[4][position+11:len(m[4])-1]+ "  >" + "<img src=\""+m[4][position+11:len(m[4])-1]+"\">" +"</a>"
   
        elif(m[4].find(".opus") >-1 or m[4].find(".mp3") >-1):
            mess = "<audio controls><source src="+ m[4][position+11:len(m[4])-1] +  " type='audio/ogg'>Your browser does not support the audio element.</audio>"
        else:
            mess = "<a href=\""+m[4][position+11:len(m[4])-1]+"\">"+m[4][position+11:len(m[4])-1]+"</a>"
        i.write(media_template.render(type=m[0], message=mess,time=m[2][0:5]))
        
    else:
        i.write(message_template.render(type=m[0], message=mess,time=m[2][0:5]))

1578 2048
225 224


In [27]:
i.write(end.render())
i.close()